In [2]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [5]:
print('x_train: ', x_train.shape)

x_train:  (50000, 32, 32, 3)


In [63]:
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
print('Noof classes:',K)
print('Noof training images:',Ntr)
print('Noof test images:',Nte)
print('Noof features:',Din)
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image

Noof classes: 2
Noof training images: 50000
Noof test images: 10000
Noof features: 3072


In [8]:
print('original labels:',y_train.shape)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
print('labels in class matrix:',y_train.shape)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)

original labels: (50000, 1)
labels in class matrix: (50000, 10)


In [10]:
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
print('x_train:', x_train.shape)
print('x_test:', x_test.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train: (50000, 3072)
x_test: (10000, 3072)


In [11]:
std=1e-5
w1 = std*np.random.randn(Din, K)
b1 = np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)

w1: (3072, 10)
b1: (10,)


In [95]:
batch_size = Ntr
iterations = 300
lr = 1.4e-2
lr_decay=0.999
reg =5e-6
loss_history = []
train_acc_history = []
val_acc_history = []
seed = 0
rng = np.random.default_rng(seed=seed)
for t in range(iterations):
    indices = np.arange(Ntr)
    #rng.shuffle(indices)
    x_train=x_train[indices]

    # Forward pass
    y_pred=np.dot(x_train,w1) + b1
    #classes=np.argmax(prediction,axis=1)
    #prediction=y_pred(classes)
    #classes=np.reshape(classes,(Ntr,1))
    #classes=tf.keras.utils.to_categorical(classes, num_classes=K)
    
    loss=1./batch_size*(np.square(y_pred-y_train).sum() + reg * np.sum(w1*w1))
    loss_history.append(loss)

    if t%10==0:
        print('iteration %d / %d : loss %f' % (t,iterations,loss))
        
    # Backward pass

    dy_pred=1./batch_size*2.0*(y_pred-y_train)
    dw1=x_train.T.dot(dy_pred) + 1./Ntr*reg * w1
    db1=dy_pred.sum(axis=0)
    #db1=np.reshape(db1,(K))
    w1 -= lr*dw1
    b1 -= lr*db1
    lr *= lr_decay






iteration 0 / 300 : loss 0.900000


KeyboardInterrupt: 

In [ ]:
# Printing accuracies and displaying w as images
f,ax=plt.subplots(1,5)

for i in range(10):
    im=w1[:,i].reshape(32,32,3)
    ax[i].imshow(im)
plt.show()




In [ ]:
train_acc = 1.0 - 1/Ntr*(np.abs(np.argmax(y_train,axis=1) - np.argmax(y_pred,axis=1))).sum()
train_acc_history.append(train_acc)
#test_acc = 1.0 - 1/Nte*(np.abs(np.argmax(y_test,axis=1) - np.argmax(y_pred,axis=1))).sum()
